In [1]:
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
})

In [3]:
import igraph as ig
import numpy as np

In [15]:
src = np.random.randint(10, size=10)
dst = np.random.randint(10, size=10)
edges = list(zip(src, dst))
edges

[(8, 2),
 (4, 7),
 (8, 8),
 (5, 9),
 (3, 3),
 (7, 7),
 (4, 1),
 (3, 4),
 (4, 8),
 (8, 9)]

In [17]:
G=ig.Graph(edges, directed=False)
layt=G.layout('kk', dim=3) 

In [27]:
N = 10
Xn=layt[:,0]
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None] 

TypeError: list indices must be integers or slices, not tuple

In [28]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             #color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               #text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
        )

data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='Les-Miserables')

In [287]:
from collections import UserDict
from itertools import islice
import plotly.plotly as py
import plotly.graph_objs as go

class UniqueDict(UserDict):
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def add(self, x):
        try:
            return self.data[x]
        except KeyError:
            n = len(self.data)
            self.data[x] = n
            return n
        
    def to_list(self):
        return sorted(self, key=self.get)
    

class RecNet():
    
    def __init__(self, seq, y_proj=None):
        edges = []
        nodes = []
        tokens = []
        token_edges = []
        unique_tokens = UniqueDict()
        seq = map(frozenset, filter(bool, seq))
        for t, tok in enumerate(seq):
            tok_id = unique_tokens.add(tok)
            
            matched = set()
            for d, tok_prev in enumerate(reversed(tokens)):
                common = tok & tok_prev
                if not common:
                    continue
                if common & matched:
                    continue
                
                edges.append((t-d-1, t, common))
                matched.update(common)
                
                if common == tok:
                    break
            
            tokens.append(tok)
            nodes.append(tok_id)
        
        self.nodes = np.array(nodes)
        self.tokens = tokens
        self.unique_tokens = np.array(unique_tokens.to_list())
        self.weights = np.array([w for _,_,w in edges])
        self.edges = np.array([(t0,t1) for t0,t1,_ in edges])
        self.edges_token_ids = self.nodes[self.edges]
        
        self.y_proj = y_proj or self.get_default_y_proj()
        
    def get_default_y_proj(self):
        #Ytok = np.random.random([len(self.unique_tokens)])
        Yn = self.nodes / len(self.unique_tokens)
        return lambda self: Yn
        
    def summary(self, verbose_level=1):
        fs_fmt = lambda fs: "{" + ','.join(fs) + "}"
        list_fmt = lambda xs: "[" + ', '.join(xs) + "]"
        if verbose_level > 1:
            print("cascade identifiers:", list_fmt(fs_fmt(t) for t in self.unique_tokens))
        print(len(self.unique_tokens), "cascade identifiers")
        print(len(self.nodes), "nodes")
        if verbose_level > 1:
            print("edges:", list_fmt(f"{s}-{d} {fs_fmt(w)}" for (s, d), w in zip(self.edges, self.weights)))
        print(len(self.edges), "edges")
        
    def plot(self, timeslice=None):

        Yn = self.y_proj(self)
        
        edge_trace = go.Scatter(
            x=[],
            y=[],
            text=[],
            line=dict(width=0.5,color='#888'),
            hoverinfo='none',
            
            mode='lines')

        nodes = zip(self.tokens, self.nodes)
        edges = zip(self.edges, self.edges_token_ids, self.weights)
        if timeslice:
            edges = islice(edges, *timeslice)
            markers = islice(markers, timeslice)
        
        edge_trace['x'] = self.edges.flatten()
        edge_trace['y'] = Yn[self.edges_token_ids].flatten()

        node_trace = go.Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            hoverinfo='text',
        )

        node_trace['x'] = np.arange(len(self.nodes))
        node_trace['y'] = Yn[self.nodes]
        node_trace['text'] = list(map(";".join, self.tokens))

        layout = dict(
            title='Recurrence network',
            xaxis=dict(
                rangeselector=dict(
                    buttons=[
                        dict(step='all')
                    ]
                ),
                rangeslider=dict(
                    visible = True,
                ),
                type='linear',
                range=[0,50],
            )
        )
        
        fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
        
        plotly.offline.iplot(fig)#, filename='basic-line')



In [288]:
import random
def seq(n):
    A = "ABCDE"
    r = np.random.random([n,2])
    m = len(A)
    for i in range(n):
        toks = []
        for j, a in enumerate(A):
            if r[i,0] < 1-j/m:
                toks.append(random.choice(A))
            if r[i,1] < j/m:
                continue
            
        yield toks

net = RecNet(seq(1000))
net.summary()
net.plot()

31 cascade identifiers
1000 nodes
1811 edges


In [232]:
# color options
"""
            marker=dict(
                showscale=True,
                # colorscale options
                #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
                #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
                #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
                colorscale='YlGnBu',
                reversescale=True,
                color=[],
                size=10,
                colorbar=dict(
                    thickness=15,
                    title='Node Connections',
                    xanchor='left',
                    titleside='right'
                ),
                line=dict(width=2)))
"""

"\n            marker=dict(\n                showscale=True,\n                # colorscale options\n                #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |\n                #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |\n                #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |\n                colorscale='YlGnBu',\n                reversescale=True,\n                color=[],\n                size=10,\n                colorbar=dict(\n                    thickness=15,\n                    title='Node Connections',\n                    xanchor='left',\n                    titleside='right'\n                ),\n                line=dict(width=2)))\n"

In [289]:
def load_tokens(filename, sep=';'):
    with open(filename, 'r') as fr:
        for line in fr.readlines():
            toks = [t.strip() for t in line.split(sep)]
            if all(toks):
                yield toks

toks = islice(load_tokens('tokens.txt'), 1000)
net = RecNet(toks)
net.summary()
net.plot()

952 cascade identifiers
1000 nodes
716 edges
